# Initialization

In [32]:
import os
import lephare as lp
import numpy as np
from matplotlib import pylab as plt

%matplotlib inline

# Create filter library

In [ ]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..')) #change to your lephare base_dir
filter_path = os.path.join(base_dir, 'simulation_catalogs/DES/DES/DES_STARCAT/WORK_COMPLETE2/filt') #paste your relative filter path
print(filter_path)
filter_rep = lp.keyword("FILTER_REP", filter_path)
filter_list = lp.keyword("FILTER_LIST",
                         "DES_filter_g.res,DES_filter_r.res,\
                         DES_filter_i.res,DES_filter_z.res,DES_filter_Y.res") #edit your filter list
#crash if add comma

filterLib = lp.Filter(config_keymap={"FILTER_REP":filter_rep,
                                   "FILTER_LIST":filter_list,
                                   "TRANS_TYPE":lp.keyword("TRANS_TYPE", "0"),
                                   "FILTER_CALIB":lp.keyword("FILTER_CALIB", "0"),
                                   "FILTER_FILE":lp.keyword("FILTER_FILE", "photozDES")})


filterLib.run()


/home/hallouin/Documents/thall_2025/photoz/lephare/training_stats/simulation_catalogs/DES/DES/DES_STARCAT/WORK_COMPLETE2/filt
# NAME                        IDENT      Lbda_mean Lbeff(Vega)       FWHM     AB-cor    TG-cor      VEGA M_sun(AB)   CALIB      Lb_eff    Fac_corr
DES_filter_g.res              1             0.4827      0.4735      0.1299   -0.0867   -0.2558  -20.7147    5.0926       0      0.4753      1.0000
DES_filter_r.res              2             0.6432      0.6343      0.1484    0.1835    0.3154  -21.6187    4.6235       0      0.6375      1.0000
DES_filter_i.res              3             0.7827      0.7749      0.1482    0.4149    0.6384  -22.2807    4.5296       0      0.7779      1.0000
DES_filter_z.res              4             0.9179      0.9139      0.1471    0.5235    0.7629  -22.7383    4.5124       0      0.9139      1.0000
DES_filter_Y.res              5             0.9900      0.9880      0.0661    0.5722    0.7890  -22.9560    4.5088       0      0.9886     

# Create SED library

In [42]:
###Buzzard

SED_path = os.path.join(base_dir, 'simulation_catalogs/buzzard_base/SEDS/updated_Buzzard_SEDs')

# SED_list = sorted([
#     f for f in os.listdir(SED_path)
#     if os.path.isfile(os.path.join(SED_path, f)) and f.endswith(".sed")
# ])

SED_list = [f'kmeansbuzzard_{i}.sed' for i in range(0,100)]

#save to file.list because we must do it like this
SED_list_path = os.path.join(SED_path, "updated_Buzzard_SEDs.list")
with open(SED_list_path, "w") as f:
    for sed_file in SED_list:
        f.write(sed_file + "\n")

SED_rep = lp.keyword("SED_REP", SED_path)
# SED_list = lp.keyword("SED_LIST",f"{SED_list}")
SED_rep

(SED_REP, /home/hallouin/Documents/thall_2025/photoz/lephare/training_stats/simulation_catalogs/buzzard_base/SEDS/updated_Buzzard_SEDs)

In [ ]:
import shutil

#Lephare cache file
LEPHARE_SED_GAL_PATH = os.path.expanduser("~/.cache/lephare/data/sed/STAR/DES")

#Create cache file in case
os.makedirs(LEPHARE_SED_GAL_PATH, exist_ok=True)

# Cpypast all file to cache
for sed_file in SED_list + ["DES_star_SEDs.list"]:
    src = os.path.join(SED_path, sed_file)
    dst = os.path.join(LEPHARE_SED_STAR_PATH, sed_file)
    shutil.copyfile(src, dst)

print(f"past {len(SED_list)} file.sed and file.list in {LEPHARE_SED_STAR_PATH}")

In [44]:
SED_list_path = "/home/hallouin/Documents/thall_2025/photoz/lephare/training_stats/simulation_catalogs/DES/DES/DES_STARCAT/WORK_COMPLETE2/lib_bin/STAR_MOD.list"

sedLib = lp.Sedtolib(config_keymap={
    # "SED_REP": SED_rep,
    "STAR_SED": lp.keyword("STAR_SED", SED_list_path),
    "STAR_FSCALE": lp.keyword("STAR_FSCALE", "3.432e-09"),
    "STAR_LIB": lp.keyword("STAR_LIB", 'DES_SEDs')},)
sedLib.run(typ="S")

#######################################
# It s translating SEDs to binary library #
# with the following options :           
# Config file     : 
# Library type     : STAR
# STAR_SED    :/home/hallouin/Documents/thall_2025/photoz/lephare/training_stats/simulation_catalogs/DES/DES/DES_STARCAT/WORK_COMPLETE2/lib_bin/STAR_MOD.list
# STAR_LIB    :DES_SEDs
# STAR_LIB doc:/home/hallouin/.cache/lephare/work/lib_bin/DES_SEDs.doc
# STAR_FSCALE :0.0000
#######################################
Number of SED in the list 135


In [ ]:
# lib_bin_dir = os.path.join(base_dir, "lib_bin")
# os.listdir(lib_bin_dir)

In [ ]:
# doc_path = os.path.join(lib_bin_dir, "buzzard_SEDs.doc")
# with open(doc_path, "r") as f:
#     for i, line in enumerate(f):
#         if i < 20:
#             print(line.strip())


# Create magnitude library

In [ ]:
os.environ["MKL_NUM_THREADS"] = "3"
# os.environ["NUMEXPR_NUM_THREADS"] = "3"


magLib = lp.MagGal(config_keymap={
    "COSMOLOGY": lp.keyword("COSMOLOGY", "70,0.3,0.7"),
    "GAL_LIB_IN": lp.keyword("GAL_LIB_IN", "buzzard_SEDs"), 
    "GAL_LIB_OUT": lp.keyword("GAL_LIB_OUT", "buzzard_mags"),
    "FILTER_FILE":lp.keyword("FILTER_FILE", "photozdc1"),
    "MAGTYPE": lp.keyword("MAGTYPE", "AB"),
    "Z_STEP": lp.keyword("Z_STEP", "0.04,0,2"),
    #"ZGRID_TYPE": lp.keyword("ZGRID_TYPE", "0"),
    "EXTINC_LAW": lp.keyword("EXTINC_LAW", "calzetti.dat"),
    "EB_V": lp.keyword("EB_V", "0.0"),
    "MOD_EXTINC": lp.keyword("MOD_EXTINC", "0"),
    "LIB_ASCII": lp.keyword("LIB_ASCII", "YES"),
    # "EM_LINES": lp.keyword("EM_LINES", "EMP_UV"),
    # "EM_DISPERSION": lp.keyword("EM_DISPERSION", "1"),
    # "ADD_DUSTEM": lp.keyword("ADD_DUSTEM", "YES"),
})
magLib.run(typ="G")